In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [26]:
#results_dir = "all_experiments"
results_dir = "../"
#sub_files = ["momentumTest", "chatAdamTests/checkpoints", "ScheduleTest_constant", "scheduleTest_all4", "1Test", "05Test", "01Test", "simpleCnnTests", "rdmExp_tests", "adrien", "adri_weights_no_reset", "TestTinyCNN", "TestTinyCnnAdam", 'TestTinyCnnAdam_all_input', "TestTinyCnnSgd_Input_all", "TestTinyCNN_every_layer_adam", "TestTinyCNN_every_layer_sgd"]
#sub_files = ["TMP_baby_stuck_noise", '1Test', '01Test']
#sub_files = ['Tests1/SGD', 'Tests1/M_SGD', 'Tests1/ADAM']
sub_files = ['EarlyTest4/ADAM', 'EarlyTest4/SGD', 'EarlyTest4/M_SGD', 'EarlyTest4/ADAM_bis']
print(len(sub_files))
json_dir = [os.path.join(results_dir, sub_file, "metrics") for sub_file in sub_files]
json_dir += [os.path.join(results_dir, sub_file, "checkpoints/metrics") for sub_file in sub_files]

4


In [27]:
json_files = []
for dir_name in json_dir:
    if not os.path.exists(dir_name):
        print(f"Directory {dir_name} does not exist.")
    else:
        for root, dirs, files in os.walk(dir_name):
            for file in files:
                if file == "epoch_metrics.jsonl":
                    json_files.append(os.path.join(root, file))
json_files

Directory ../EarlyTest4/ADAM/metrics does not exist.
Directory ../EarlyTest4/SGD/metrics does not exist.
Directory ../EarlyTest4/M_SGD/metrics does not exist.
Directory ../EarlyTest4/ADAM_bis/metrics does not exist.


['../EarlyTest4/ADAM/checkpoints/metrics/simple_cnn_none_20250613_033614/epoch_metrics.jsonl',
 '../EarlyTest4/ADAM/checkpoints/metrics/tiny_cnn_none_20250613_084349/epoch_metrics.jsonl',
 '../EarlyTest4/ADAM/checkpoints/metrics/baby_cnn_none_20250613_081415/epoch_metrics.jsonl',
 '../EarlyTest4/ADAM/checkpoints/metrics/tiny_cnn_none_20250613_082905/epoch_metrics.jsonl',
 '../EarlyTest4/ADAM/checkpoints/metrics/tiny_cnn_none_20250613_045155/epoch_metrics.jsonl',
 '../EarlyTest4/ADAM/checkpoints/metrics/resnet18_none_20250613_050252/epoch_metrics.jsonl',
 '../EarlyTest4/ADAM/checkpoints/metrics/simple_cnn_none_20250613_054621/epoch_metrics.jsonl',
 '../EarlyTest4/ADAM/checkpoints/metrics/simple_cnn_none_20250613_071339/epoch_metrics.jsonl',
 '../EarlyTest4/ADAM/checkpoints/metrics/baby_cnn_none_20250613_022257/epoch_metrics.jsonl',
 '../EarlyTest4/ADAM/checkpoints/metrics/simple_cnn_none_20250613_053124/epoch_metrics.jsonl',
 '../EarlyTest4/ADAM/checkpoints/metrics/resnet18_none_2025061

In [28]:
def flatten_config(config):
    flat = {}
    for k, v in config.items():
        if isinstance(v, list):
            flat[k] = str(v)  # convertir liste en string
        else:
            flat[k] = v
    return flat

In [29]:
all_last_lines_data = []

for jsonl_file_path in json_files:

    dir_name = os.path.dirname(jsonl_file_path)

    print(f"Traitement du fichier : {jsonl_file_path}")
    try:
        # Lire l'intégralité du fichier JSONL avec Pandas
        # C'est la méthode la plus fiable et souvent assez rapide pour ça.
        df_file = pd.read_json(jsonl_file_path, lines=True, encoding='utf-8')

        df_config = pd.read_json(os.path.join(dir_name, 'config.json'), encoding='utf-8')

        if not df_config.empty:
            config = flatten_config(df_config)
        else:
            print(f"  -> Le fichier config.json dans {dir_name} est vide ou n'a pas de lignes valides.")

        if not df_file.empty:
            # Sélectionner la dernière ligne du DataFrame
            last_line_series = df_file.iloc[-1]

            merged_data_for_row = {
                config,
                last_line_series
            }


            # Convertir la Series en dictionnaire et l'ajouter à notre liste
            all_last_lines_data.append(merged_data_for_row.to_dict())
            print(f"  -> Dernière ligne trouvée : {last_line_series['epoch']}, {last_line_series['val_acc']}")
        else:
            print(f"  -> Le fichier {jsonl_file_path} est vide ou n'a pas de lignes valides.")

    except Exception as e:
        print(f"Erreur lors de la lecture de {jsonl_file_path} : {e}")

# Créer le DataFrame final à partir de toutes les dernières lignes collectées
if all_last_lines_data:
    final_df_last_lines = pd.DataFrame(all_last_lines_data)
    print("\n--- DataFrame des dernières lignes ---")
    print(final_df_last_lines)
    print(f"\nTotal des fichiers traités : {len(json_files)}")
    print(f"Total des dernières lignes collectées : {len(final_df_last_lines)}")
else:
    print("Aucune dernière ligne n'a pu être collectée.")

Traitement du fichier : ../EarlyTest4/ADAM/checkpoints/metrics/simple_cnn_none_20250613_033614/epoch_metrics.jsonl
Erreur lors de la lecture de ../EarlyTest4/ADAM/checkpoints/metrics/simple_cnn_none_20250613_033614/epoch_metrics.jsonl : If using all scalar values, you must pass an index
Traitement du fichier : ../EarlyTest4/ADAM/checkpoints/metrics/tiny_cnn_none_20250613_084349/epoch_metrics.jsonl
Erreur lors de la lecture de ../EarlyTest4/ADAM/checkpoints/metrics/tiny_cnn_none_20250613_084349/epoch_metrics.jsonl : If using all scalar values, you must pass an index
Traitement du fichier : ../EarlyTest4/ADAM/checkpoints/metrics/baby_cnn_none_20250613_081415/epoch_metrics.jsonl
Erreur lors de la lecture de ../EarlyTest4/ADAM/checkpoints/metrics/baby_cnn_none_20250613_081415/epoch_metrics.jsonl : If using all scalar values, you must pass an index
Traitement du fichier : ../EarlyTest4/ADAM/checkpoints/metrics/tiny_cnn_none_20250613_082905/epoch_metrics.jsonl
Erreur lors de la lecture de ..

In [30]:
import pandas as pd
import os
import json

# Assurez-vous que cette fonction est bien définie et disponible dans votre script
# C'est une implémentation générique, utilisez la vôtre si elle est différente.
def flatten_config(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_config(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Votre liste de fichiers JSONL doit être définie avant cette boucle, par exemple:
# json_files = ["../EarlyTest4/M_SGD/checkpoints/metrics/baby_cnn_none_20250613_092520/epoch_metrics.jsonl",
#               "../EarlyTest4/M_SGD/checkpoints/metrics/baby_cnn_none_20250613_043624/epoch_metrics.jsonl"]
# Ou collectés via os.walk comme précédemment suggéré.

all_last_lines_data = []

for jsonl_file_path in json_files:
    dir_name = os.path.dirname(jsonl_file_path)
    run_name = os.path.basename(dir_name) # Récupérer le nom du dossier (nom du run)

    print(f"Traitement du fichier : {jsonl_file_path}")

    # Initialisation des données pour ce run en cas d'erreur de lecture
    flat_run_config = {}
    last_metric_line_dict = {}

    try:
        # --- Lecture et aplatissement du fichier config.json ---
        config_file_path = os.path.join(dir_name, 'config.json')
        try:
            with open(config_file_path, 'r', encoding='utf-8') as f:
                # Utiliser json.load() pour lire un fichier JSON unique
                run_config_data = json.load(f)
                flat_run_config = flatten_config(run_config_data)
        except FileNotFoundError:
            print(f"  AVERTISSEMENT: Fichier config.json non trouvé dans {dir_name}. La configuration sera vide.")
        except json.JSONDecodeError as e:
            print(f"  ERREUR: Fichier config.json malformé dans {dir_name}: {e}. La configuration sera vide.")
        except Exception as e:
            print(f"  ERREUR inattendue lors de la lecture du config.json dans {dir_name}: {e}. La configuration sera vide.")


        # --- Lecture de la dernière ligne du fichier epoch_metrics.jsonl ---
        try:
            # pd.read_json est utilisé pour les fichiers JSONL (lines=True)
            df_file = pd.read_json(jsonl_file_path, lines=True, encoding='utf-8')

            if not df_file.empty:
                # Sélectionner la dernière ligne et la convertir en dictionnaire
                last_metric_line_dict = df_file.iloc[-1].to_dict()
                print(f"  -> Dernière ligne de métriques trouvée : Epoch {last_metric_line_dict.get('epoch', 'N/A')}, Val Acc {last_metric_line_dict.get('val_acc', 'N/A')}")
            else:
                print(f"  -> Le fichier {jsonl_file_path} est vide ou n'a pas de lignes valides. Pas de métriques collectées.")

        # Gestion spécifique des erreurs de Pandas pour les fichiers JSONL
        except pd.errors.ParserError as e:
            print(f"  ERREUR ParserError lors de la lecture de {jsonl_file_path} : {e}. Fichier potentiellement malformé ou non-JSON.")
        except ValueError as e:
            print(f"  ERREUR ValueError (fichier vide/invalide) lors de la lecture de {jsonl_file_path} : {e}. Le fichier JSONL est probablement vide ou ne contient aucun objet JSON valide.")
        except Exception as e:
            print(f"  ERREUR inattendue lors de la lecture des métriques de {jsonl_file_path} : {e}.")

        # --- Fusion des données de configuration et de métriques ---
        # Utiliser l'opérateur de fusion de dictionnaires (**)
        merged_data_for_row = {
            **flat_run_config,          # Ajoute toutes les clés/valeurs de la config
            **last_metric_line_dict     # Ajoute toutes les clés/valeurs des métriques (écrase si doublon)
        }
        # Ajouter le nom du run pour identification
        merged_data_for_row['run_name'] = run_name

        # Ajouter la ligne fusionnée à la liste principale si elle contient des données
        if merged_data_for_row: # Vérifie si le dictionnaire n'est pas vide (au moins run_name sera là)
            all_last_lines_data.append(merged_data_for_row)
        else:
            print(f"  AVERTISSEMENT: Aucune donnée (config ou métriques) n'a pu être collectée pour le run '{run_name}'. Ligne non ajoutée.")

    except Exception as e:
        # Cette exception attrape les erreurs qui n'ont pas été gérées spécifiquement ci-dessus
        print(f"ERREUR GÉNÉRALE lors du traitement de {jsonl_file_path} : {e}")

# Créer le DataFrame final à partir de toutes les dernières lignes collectées et fusionnées
if all_last_lines_data:
    final_df_last_lines = pd.DataFrame(all_last_lines_data)
    print("\n--- DataFrame des dernières lignes fusionnées ---")
    print(final_df_last_lines)
    print(f"\nTotal des fichiers traités : {len(json_files)}")
    print(f"Total des lignes de résultats collectées : {len(final_df_last_lines)}")
else:
    print("Aucune dernière ligne fusionnée n'a pu être collectée.")

Traitement du fichier : ../EarlyTest4/ADAM/checkpoints/metrics/simple_cnn_none_20250613_033614/epoch_metrics.jsonl
  -> Dernière ligne de métriques trouvée : Epoch 300, Val Acc 0.8740000000000001
Traitement du fichier : ../EarlyTest4/ADAM/checkpoints/metrics/tiny_cnn_none_20250613_084349/epoch_metrics.jsonl
  -> Dernière ligne de métriques trouvée : Epoch 300, Val Acc 0.7098
Traitement du fichier : ../EarlyTest4/ADAM/checkpoints/metrics/baby_cnn_none_20250613_081415/epoch_metrics.jsonl
  -> Dernière ligne de métriques trouvée : Epoch 300, Val Acc 0.5508000000000001
Traitement du fichier : ../EarlyTest4/ADAM/checkpoints/metrics/tiny_cnn_none_20250613_082905/epoch_metrics.jsonl
  -> Dernière ligne de métriques trouvée : Epoch 300, Val Acc 0.7096
Traitement du fichier : ../EarlyTest4/ADAM/checkpoints/metrics/tiny_cnn_none_20250613_045155/epoch_metrics.jsonl
  -> Dernière ligne de métriques trouvée : Epoch 90, Val Acc 0.7858
Traitement du fichier : ../EarlyTest4/ADAM/checkpoints/metrics/re

In [31]:
final_df_last_lines

,data_dir,checkpoint_name,download_weights,classifier,batch_size,max_epochs,num_workers,gpu_id,learning_rate,weight_decay,...,detailed_metrics,epoch,train_loss,train_acc,val_loss,val_acc,training_per_class_metrics,validation_per_class_metrics,timestamp,run_name
0,./data/cifar10,simple_cnn_1.0_adam_baseline.pth,False,simple_cnn,256,300,6,0,0.000010,0.0001,...,False,300,0.001762,0.999760,0.786366,0.8740,NaN,"{'airplane': {'accuracy': 0.902, 'avg_loss': 0...",2025-06-13 02:21:58.694930077,simple_cnn_none_20250613_033614
1,./data/cifar10,tiny_cnn_1.0_adam_none_0.01_cosine.pth,False,tiny_cnn,256,300,6,0,0.000010,0.0001,...,False,300,0.874570,0.698878,0.838895,0.7098,NaN,"{'airplane': {'accuracy': 0.742, 'avg_loss': 0...",2025-06-13 06:58:14.174057961,tiny_cnn_none_20250613_084349
2,./data/cifar10,baby_cnn_1.0_adam_none_0.01_cosine.pth,False,baby_cnn,256,300,6,0,0.000010,0.0001,...,False,300,1.381384,0.517368,1.296264,0.5508,NaN,"{'airplane': {'accuracy': 0.636, 'avg_loss': 1...",2025-06-13 06:28:58.393248320,baby_cnn_none_20250613_081415
3,./data/cifar10,tiny_cnn_1.0_adam_none_0.01_cosine.pth,False,tiny_cnn,256,300,6,0,0.000010,0.0001,...,False,300,0.874331,0.699139,0.838722,0.7096,NaN,"{'airplane': {'accuracy': 0.734, 'avg_loss': 0...",2025-06-13 06:43:42.148478270,tiny_cnn_none_20250613_082905
4,./data/cifar10,tiny_cnn_1.0_adam_none_0.01_cosine.pth,False,tiny_cnn,256,300,6,0,0.000032,0.0001,...,False,90,0.591227,0.794832,0.625020,0.7858,NaN,"{'airplane': {'accuracy': 0.834, 'avg_loss': 0...",2025-06-13 02:56:17.296729803,tiny_cnn_none_20250613_045155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,./data/cifar10,tiny_cnn_1.0_adam_none_0.01_cosine.pth,False,tiny_cnn,256,300,6,0,0.000662,0.0001,...,False,18,NaN,0.099960,NaN,0.1000,NaN,"{'airplane': {'accuracy': 1.0, 'avg_loss': Non...",2025-06-13 02:26:25.150816917,tiny_cnn_none_20250613_042530
86,./data/cifar10,simple_cnn_1.0_adam_none_0.01_cosine.pth,False,simple_cnn,256,300,6,0,0.098809,0.0001,...,False,2,NaN,0.099940,NaN,0.1000,NaN,"{'airplane': {'accuracy': 1.0, 'avg_loss': Non...",2025-06-13 03:05:52.558168650,simple_cnn_none_20250613_050531
87,./data/cifar10,baby_cnn_1.0_adam_baseline.pth,False,baby_cnn,256,300,6,0,0.000366,0.0001,...,False,147,NaN,0.099980,NaN,0.1000,NaN,"{'airplane': {'accuracy': 1.0, 'avg_loss': Non...",2025-06-13 02:37:03.502203226,baby_cnn_none_20250613_043005
88,./data/cifar10,tiny_cnn_1.0_adam_none_0.01_cosine.pth,False,tiny_cnn,256,300,6,0,0.010672,0.0001,...,False,179,NaN,0.100100,NaN,0.1000,NaN,"{'airplane': {'accuracy': 1.0, 'avg_loss': Non...",2025-06-13 03:03:10.019948483,tiny_cnn_none_20250613_045431


In [32]:
#final_df_last_lines.sort_values(by='final_val_acc', ascending=False, inplace=True)
sub_df = final_df_last_lines.copy()
sub_df.drop(columns=['data_dir', 'checkpoint_name', 'download_weights', 'model'], inplace=True)
sub_df.drop(columns=['batch_size', 'max_epochs', 'num_workers', 'gpu_id'], inplace=True)
sub_df.drop(columns=['test_phase','noise_type'], inplace=True)
sub_df.drop(columns=['noise_layer', 'weight_decay'], inplace=True)

sub_df['noise_distribution'] = sub_df.apply(lambda row: 'baseline' if row['noise_schedule'] =='constant' else row['noise_distribution'],axis=1)

sub_df.drop(columns=['noise_schedule', 'noise_magnitude'], inplace=True)
sub_df

,classifier,learning_rate,noise_distribution,optimizer,momentum,beta1,beta2,permanent,noise_during_stuck_only,patience,...,detailed_metrics,epoch,train_loss,train_acc,val_loss,val_acc,training_per_class_metrics,validation_per_class_metrics,timestamp,run_name
0,simple_cnn,0.000010,baseline,adam,0.0,0.9,0.999,False,False,5,...,False,300,0.001762,0.999760,0.786366,0.8740,NaN,"{'airplane': {'accuracy': 0.902, 'avg_loss': 0...",2025-06-13 02:21:58.694930077,simple_cnn_none_20250613_033614
1,tiny_cnn,0.000010,uniform,adam,0.0,0.9,0.999,True,True,10,...,False,300,0.874570,0.698878,0.838895,0.7098,NaN,"{'airplane': {'accuracy': 0.742, 'avg_loss': 0...",2025-06-13 06:58:14.174057961,tiny_cnn_none_20250613_084349
2,baby_cnn,0.000010,uniform,adam,0.0,0.9,0.999,True,True,10,...,False,300,1.381384,0.517368,1.296264,0.5508,NaN,"{'airplane': {'accuracy': 0.636, 'avg_loss': 1...",2025-06-13 06:28:58.393248320,baby_cnn_none_20250613_081415
3,tiny_cnn,0.000010,gaussian,adam,0.0,0.9,0.999,True,True,10,...,False,300,0.874331,0.699139,0.838722,0.7096,NaN,"{'airplane': {'accuracy': 0.734, 'avg_loss': 0...",2025-06-13 06:43:42.148478270,tiny_cnn_none_20250613_082905
4,tiny_cnn,0.000032,gaussian,adam,0.0,0.9,0.999,True,True,10,...,False,90,0.591227,0.794832,0.625020,0.7858,NaN,"{'airplane': {'accuracy': 0.834, 'avg_loss': 0...",2025-06-13 02:56:17.296729803,tiny_cnn_none_20250613_045155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,tiny_cnn,0.000662,uniform,adam,0.0,0.9,0.999,True,True,10,...,False,18,NaN,0.099960,NaN,0.1000,NaN,"{'airplane': {'accuracy': 1.0, 'avg_loss': Non...",2025-06-13 02:26:25.150816917,tiny_cnn_none_20250613_042530
86,simple_cnn,0.098809,gaussian,adam,0.0,0.9,0.999,True,True,10,...,False,2,NaN,0.099940,NaN,0.1000,NaN,"{'airplane': {'accuracy': 1.0, 'avg_loss': Non...",2025-06-13 03:05:52.558168650,simple_cnn_none_20250613_050531
87,baby_cnn,0.000366,baseline,adam,0.0,0.9,0.999,False,False,5,...,False,147,NaN,0.099980,NaN,0.1000,NaN,"{'airplane': {'accuracy': 1.0, 'avg_loss': Non...",2025-06-13 02:37:03.502203226,baby_cnn_none_20250613_043005
88,tiny_cnn,0.010672,uniform,adam,0.0,0.9,0.999,True,True,10,...,False,179,NaN,0.100100,NaN,0.1000,NaN,"{'airplane': {'accuracy': 1.0, 'avg_loss': Non...",2025-06-13 03:03:10.019948483,tiny_cnn_none_20250613_045431


In [61]:
df_selection = sub_df[['classifier', 'noise_distribution', 'optimizer', 'momentum', 'train_acc', 'val_acc', 'train_loss', 'val_loss', 'learning_rate']].copy()

df_selection.sort_values(by='val_acc', ascending=False, inplace=True)
df_selection = df_selection[df_selection['classifier'] == 'baby_cnn']
df_selection = df_selection[df_selection['optimizer'] == 'sgd']
df_selection = df_selection[df_selection['momentum'] == 0.0]
#df_selection = df_selection[df_selection['noise_distribution'] == 'baseline']
df_selection

,classifier,noise_distribution,optimizer,momentum,train_acc,val_acc,train_loss,val_loss,learning_rate
43,baby_cnn,baseline,sgd,0.0,0.545052,0.5770,1.306659,1.223467,0.000010
40,baby_cnn,gaussian,sgd,0.0,0.544050,0.5748,1.308923,1.226179,0.000010
25,baby_cnn,uniform,sgd,0.0,0.543530,0.5748,1.309079,1.227084,0.000010
41,baby_cnn,uniform,sgd,0.0,0.155128,0.1194,2.825798,2.494512,0.044705
27,baby_cnn,gaussian,sgd,0.0,0.168930,0.1124,2.895783,2.594632,0.044705


In [34]:
#df["best_val_acc"] = df["val_acc"].apply(lambda x : max(x) if isinstance(x, list) else x)
#df["last_val_acc"] = df["val_acc"].apply(lambda x : x[-1] if isinstance(x, list) else x)
#df["best_train_acc"] = df["train_acc"].apply(lambda x : max(x) if isinstance(x, list) else x)
#df["last_train_acc"] = df["train_acc"].apply(lambda x : x[-1] if isinstance(x, list) else x)
#df["train_last_is_best"] = df["last_train_acc"] == df["best_train_acc"]
#df["val_last_is_best"] = df["last_val_acc"] == df["best_val_acc"]
#df

In [35]:
constant_cols = []
for col in df.columns:
    try:
        if df[col].nunique() == 1:
            constant_cols.append(col)
    except TypeError:
        # ignorer colonnes non hashables
        pass

print("Colonnes constantes :", constant_cols)


NameError: name 'df' is not defined

In [15]:
df

""


In [13]:
#sub_df = df.copy()
#sub_df.drop(columns=['data_dir', 'checkpoint_name', 'download_weights', 'model'], inplace=True)
#sub_df.drop(columns=['batch_size', 'max_epochs', 'num_workers', 'gpu_id'], inplace=True)
#sub_df.drop(columns=['learning_rate', 'test_phase',], inplace=True)
#sub_df.drop(columns=['train_acc', 'val_acc', 'noise_metrics'], inplace=True)
#sub_df
#pass

KeyError: "['data_dir', 'checkpoint_name', 'download_weights', 'model'] not found in axis"

In [14]:
cols = ['last_val_acc', 'classifier', 'noise_type', 'optimizer']

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_sorted = df.sort_values(by=cols, ascending=False)
df_sorted

KeyError: 'last_val_acc'

In [22]:
df_weight = df_sorted[(df_sorted['noise_type'] == 'weight')]
df_weight = df_weight.sort_values('noise_layer', ascending=False)

In [12]:
df_exp = df_sorted[(df_sorted['noise_schedule']=='exponential') | (df_sorted['noise_schedule'] == 'constant')]
df_exp = df_exp[df_exp['classifier'] == 'resnet18']
cols = ['noise_type','noise_layer', 'momentum']
df_exp.sort_values(cols, ascending=False)

,classifier,weight_decay,noise_type,noise_magnitude,noise_schedule,noise_layer,noise_distribution,optimizer,momentum,beta1,beta2,permanent,noise_during_stuck_only,patience,subset,best_val_acc,last_val_acc,best_train_acc,last_train_acc,train_last_is_best,val_last_is_best


In [14]:
df_no_momentum = df_exp[df_exp['momentum']==0.0]
df_no_momentum

,classifier,weight_decay,noise_type,noise_magnitude,noise_schedule,noise_layer,noise_distribution,optimizer,momentum,beta1,beta2,subset,noise_layers,best_val_acc,last_val_acc,best_train_acc,last_train_acc,train_last_is_best,val_last_is_best
4,resnet18,0.001,weight,0.01,exponential,['bias'],gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.855569,0.853365,0.926783,0.926002,False,False
22,resnet18,0.001,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.849860,0.848758,0.901362,0.901022,False,False
3,resnet18,0.001,weight,0.01,exponential,['conv1'],gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.775441,0.775441,0.808694,0.808694,True,True
2,resnet18,0.001,weight,0.01,exponential,['bias'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.765224,0.762119,0.862430,0.862430,True,False
25,resnet18,0.001,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.764123,0.746995,0.828325,0.828325,True,False
76,resnet18,0.001,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.764123,0.746995,0.828325,0.828325,True,False
8,resnet18,0.001,weight,0.01,exponential,['conv1'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.702123,0.696715,0.748898,0.748898,True,False
75,resnet18,0.001,weight,0.01,exponential,['conv1'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.702123,0.696715,0.748898,0.748898,True,False
7,resnet18,0.001,none,0.01,constant,None,gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.852264,0.851462,0.926583,0.926362,False,False
13,resnet18,0.001,none,0.01,constant,None,gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.850361,0.850361,0.927364,0.927364,True,True


In [22]:
df_tinyCNN = df_sorted[df_sorted['classifier'] == 'tiny_cnn']
df_tinyCNN = df_tinyCNN[(df_tinyCNN['momentum'] == 0)|((df_tinyCNN['beta1'] == 0))]
df_tinyCNN = df_tinyCNN[df_tinyCNN['noise_type'] == 'gradient']
df_tinyCNN = df_tinyCNN[df_tinyCNN['noise_schedule'] == 'constant']
df_tinyCNN.sort_values('best_val_acc', ascending=False)




,classifier,weight_decay,noise_type,noise_magnitude,noise_schedule,noise_layer,noise_distribution,optimizer,momentum,beta1,beta2,subset,noise_layers,best_val_acc,last_val_acc,best_train_acc,last_train_acc,train_last_is_best,val_last_is_best
129,tiny_cnn,0.001,gradient,0.05,exponential,['layer4.'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.840345,0.839543,0.865705,0.865705,True,False
284,tiny_cnn,0.001,gradient,0.01,exponential,None,gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.839844,0.838842,0.870693,0.869371,False,False
239,tiny_cnn,0.001,gradient,0.01,exponential,['none'],gaussian,adam,0.9,0.0,0.999,1.0,NaN,0.839143,0.838241,0.867969,0.867969,True,False
240,tiny_cnn,0.001,gradient,0.01,exponential,['none'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.838842,0.838241,0.867428,0.866426,False,False
196,tiny_cnn,0.001,gradient,0.05,exponential,['none'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.838442,0.837941,0.868770,0.868770,True,False
138,tiny_cnn,0.001,gradient,0.05,exponential,['bias'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.838241,0.837941,0.865745,0.865745,True,False
153,tiny_cnn,0.001,gradient,0.05,exponential,['conv1'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.837941,0.835938,0.864944,0.864944,True,False
219,tiny_cnn,0.001,gradient,0.05,exponential,['none'],gaussian,adam,0.9,0.0,0.999,1.0,NaN,0.836338,0.834435,0.867889,0.867889,True,False
142,tiny_cnn,0.001,gradient,0.01,exponential,['conv1'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.835837,0.834736,0.865365,0.863582,False,False
289,tiny_cnn,0.001,gradient,0.01,exponential,None,gaussian,adam,0.9,0.0,0.999,1.0,NaN,0.835337,0.834635,0.869371,0.869371,True,False
